In [2]:
!pip install -q unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.3/351.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 23.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 23.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 6.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 970.4 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.4/170.4 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.

In [4]:
!pip install -q peft

In [5]:
!pip install -q accelerate bitsandbytes

In [6]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.5 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
# ============================================================
# 1. Set Kaggle dataset path
# ============================================================
ADAPTER_DIR = "/kaggle/input/my-finetuned-llm"   # Replace with your MAK if needed

In [ ]:
from unsloth import FastLanguageModel
from peft import PeftModel
import torch

# ============================================================
# 2. Load 4-bit base model
# ============================================================
BASE_ID = "unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    BASE_ID,
    max_seq_length = 2048,
    load_in_4bit = True,
)
print("=" * 50)
print("Base model loaded")
print("=" * 50)

# ============================================================
# 3. Load LoRA adapter
# ============================================================
model = PeftModel.from_pretrained(model, ADAPTER_DIR)
print("=" * 50)
print("LoRA loaded")
print("=" * 50)

# ============================================================
# 4. Switch to inference mode
# ============================================================
FastLanguageModel.for_inference(model)
print("=" * 50)
print("Model ready for inference")
print("=" * 50)

model.eval()
print("\n\n")
print("=" * 50)
print("Fine-tuned model is ready!")
print("=" * 50)

In [8]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))
print("Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
print("Reserved:", torch.cuda.memory_reserved() / 1024**2, "MB")


CUDA available: True
GPU name: Tesla T4
Allocated: 5877.8994140625 MB
Reserved: 6044.0 MB


In [10]:
import gradio as gr
import torch
import re
from threading import Lock

# -------------------------
# YOUR MODEL + TOKENIZER
# -------------------------
tok = tokenizer
mdl = model
mdl.eval()

gen_lock = Lock()

# -------------------------
# Format the model output
# -------------------------
def format_output(raw):
    sections = {
        "verdict": r"(?i)verdict[:\-]\s*(.*)",
        "score": r"(?i)score[:\-]\s*([0-9\.]+)",
        "benefits": r"(?i)benefits?[:\-](.*?)(?=risks?:|overall|conclusion|$)",
        "risks": r"(?i)risks?[:\-](.*?)(?=overall|conclusion|$)",
        "overall": r"(?i)(overall|rationale|analysis)[:\-](.*)",
    }

    formatted = {}
    for key, pattern in sections.items():
        m = re.search(pattern, raw, re.S)
        if m:
            text = m.group(1).strip()
        else:
            text = ""
        formatted[key] = text

    return formatted


# -------------------------
# Evaluation Function
# -------------------------
history = []

def evaluate_story(story, context):
    with gen_lock:
        if not story or story.strip() == "":
            return "Please enter a story.", 0, history

        template = """Below is an instruction that describes a task.

### Instruction:
Evaluate whether the following idea uplifts humanity. Provide:
- Verdict (Yes/No or Positive/Negative)
- Score from 0 to 1
- Benefits
- Risks
- Overall rationale
Be objective and specific.

### Input:
{story}

### Additional Context:
{context}

### Response:
"""

        prompt = template.format(
            story = story.strip(),
            context = context.strip() if context else "None"
        )

        inputs = tok(prompt, return_tensors="pt").to("cuda")

        outputs = mdl.generate(
            **inputs,
            max_new_tokens = 400,
            temperature   = 0.4,
            top_p         = 0.95,
            do_sample     = True,
        )

        raw_text = tok.decode(outputs[0], skip_special_tokens=True)

        # Clean prefix if exists
        if "### Response:" in raw_text:
            raw_text = raw_text.split("### Response:")[1].strip()

        parsed = format_output(raw_text)

        # Score handling
        try:
            score = float(parsed["score"])
        except:
            score = 0

        # Save history
        history.append({
            "story": story,
            "score": score,
            "verdict": parsed["verdict"],
            "output": raw_text[:600] + ("..." if len(raw_text) > 600 else "")
        })

        display_text = f"""
### Verdict
{parsed["verdict"]}

### Score
{parsed["score"]}

### Benefits
{parsed["benefits"]}

### Risks
{parsed["risks"]}

### Overall Rationale
{parsed["overall"]}
""".strip()

        return display_text, score, history


# -------------------------
# Gradio UI
# -------------------------
with gr.Blocks() as demo:
    gr.Markdown("# 🌍 Humanity Uplift Evaluator — Kaggle Demo")
    gr.Markdown("Enter an idea / story. The model evaluates whether it uplifts humanity.")

    with gr.Row():
        story = gr.Textbox(
            label="Story / Idea",
            placeholder="Enter the narrative or idea here...",
            lines=8
        )
        context = gr.Textbox(
            label="Additional Context (optional)",
            placeholder="Studio mission, values, situation...",
            lines=4
        )

    btn = gr.Button("Evaluate", variant="primary")

    with gr.Row():
        output = gr.Markdown(label="Model Output")
        score_bar = gr.Slider(label="Score (0 to 1)", minimum=0, maximum=1, value=0, interactive=False)

    history_box = gr.JSON(label="History (previous evaluations)")

    btn.click(
        evaluate_story,
        inputs=[story, context],
        outputs=[output, score_bar, history_box]
    )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://d01d0940f230fae486.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Make sure if Gradio is working as expected

# After running Gradio, run:
import torch

print("GPU:", torch.cuda.get_device_name(0))
print("Memory allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")